# Train on accident and person data


In [1]:
%reset 
%matplotlib inline
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import xgboost as xgb
from numpy import nan as NA
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score as auc

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


# Load data and make labels consistent

In [2]:
print("Load labels")
label_file = pd.read_csv("./train/labels_ext.csv", index_col=0)

Load labels


In [3]:
Xtrain = pd.read_csv("./train/joint_accident_person_train_ext.csv", index_col=0)
n_train, n_dim = Xtrain.shape
print([n_train, n_dim])

[708569, 260]


In [4]:
temp = pd.merge(Xtrain, label_file, on=['ID'], how='inner')

In [5]:
y = temp['DRUNK_DR'].apply(lambda x: x*1 ).values

In [6]:
%xdel temp

In [7]:
Xtrain.drop('ID', axis= 1, inplace=True)
print("Splitting data")

Splitting data


In [8]:
Xtrain_c, Xeval, Ytrain_c, Yeval = train_test_split(Xtrain.values, y, test_size = 0.3)

In [9]:
%xdel Xtrain
%xdel label_file

In [10]:
%whos

Variable           Type        Data/Info
----------------------------------------
NA                 float       nan
Xeval              ndarray     212571x259: 55055889 elems, type `float64`, 440447112 bytes (420.04309844970703 Mb)
Xtrain_c           ndarray     495998x259: 128463482 elems, type `float64`, 1027707856 bytes (980.0985870361328 Mb)
Yeval              ndarray     212571: 212571 elems, type `int64`, 1700568 bytes (1.6217880249023438 Mb)
Ytrain_c           ndarray     495998: 495998 elems, type `int64`, 3967984 bytes (3.7841644287109375 Mb)
auc                function    <function roc_auc_score at 0x7f2f7d8e1ae8>
n_dim              int         260
n_train            int         708569
np                 module      <module 'numpy' from '/ho<...>kages/numpy/__init__.py'>
os                 module      <module 'os' from '/home/<...>da3/lib/python3.5/os.py'>
pd                 module      <module 'pandas' from '/h<...>ages/pandas/__init__.py'>
plt                module      <mo

In [11]:
print("Load data into sparse matrix")
dtrain = xgb.DMatrix(data=Xtrain_c, missing = NA, label= Ytrain_c)
deval  = xgb.DMatrix(data=Xeval, missing = NA, label = Yeval)
print("Specifying the parameters ... ")

Load data into sparse matrix
Specifying the parameters ... 


In [14]:
%xdel Xtrain_c
%xdel Xeval
%whos

NameError: name 'Xtrain_c' is not defined
NameError: name 'Xeval' is not defined
Variable           Type        Data/Info
----------------------------------------
NA                 float       nan
Yeval              ndarray     212571: 212571 elems, type `int64`, 1700568 bytes (1.6217880249023438 Mb)
Ytrain_c           ndarray     495998: 495998 elems, type `int64`, 3967984 bytes (3.7841644287109375 Mb)
auc                function    <function roc_auc_score at 0x7f2f7d8e1ae8>
deval              DMatrix     <xgboost.core.DMatrix object at 0x7f2f7c8bc0b8>
dtrain             DMatrix     <xgboost.core.DMatrix object at 0x7f2f7c8bc080>
n_dim              int         260
n_train            int         708569
np                 module      <module 'numpy' from '/ho<...>kages/numpy/__init__.py'>
os                 module      <module 'os' from '/home/<...>da3/lib/python3.5/os.py'>
param              dict        n=10
pd                 module      <module 'pandas' from '/h<...>ages/pandas/__in

In [15]:
param = {'max_depth': 12,
         'eta': 0.02,
         'subsample': 0.7,
         'colsample_bytree': 0.8,
         'silent': 0,
         'eval_metric': 'auc',
         'alpha': 0,
         'lambda': 1,
         'nthread': 8,
         'objective': 'binary:logistic'}

In [16]:
watchlist = [(deval, 'eval'), (dtrain, 'train')]
num_round = 520
print("Training ... ")

Training ... 


In [18]:
bst = xgb.train(param, dtrain, num_round, watchlist)

[0]	eval-auc:0.816793	train-auc:0.831687
[1]	eval-auc:0.825441	train-auc:0.840354
[2]	eval-auc:0.829731	train-auc:0.845102
[3]	eval-auc:0.832145	train-auc:0.847612
[4]	eval-auc:0.833571	train-auc:0.849153
[5]	eval-auc:0.834688	train-auc:0.850574
[6]	eval-auc:0.835894	train-auc:0.852529
[7]	eval-auc:0.836466	train-auc:0.853303
[8]	eval-auc:0.837231	train-auc:0.854312
[9]	eval-auc:0.837713	train-auc:0.854804
[10]	eval-auc:0.838087	train-auc:0.855431
[11]	eval-auc:0.838762	train-auc:0.856217
[12]	eval-auc:0.838954	train-auc:0.856522
[13]	eval-auc:0.839272	train-auc:0.856997
[14]	eval-auc:0.839488	train-auc:0.857295
[15]	eval-auc:0.839868	train-auc:0.857791
[16]	eval-auc:0.840244	train-auc:0.858400
[17]	eval-auc:0.840450	train-auc:0.858638
[18]	eval-auc:0.841010	train-auc:0.859184
[19]	eval-auc:0.841302	train-auc:0.859498
[20]	eval-auc:0.841563	train-auc:0.859941
[21]	eval-auc:0.841915	train-auc:0.860492
[22]	eval-auc:0.842331	train-auc:0.861009
[23]	eval-auc:0.842436	train-auc:0.861252
[2

In [19]:
print("Saving the model")
bst.save_model('./models/xgb_acc_per.model')
bst.dump_model('./models/xgb_raw_acc_per.txt')

dtrain.save_binary('./models/binary/dtrain.buffer')
deval.save_binary('./models/binary/deval.buffer')

%xdel dtrain
%xdel deval

Saving the model


In [20]:
%whos

Variable           Type        Data/Info
----------------------------------------
NA                 float       nan
Yeval              ndarray     212571: 212571 elems, type `int64`, 1700568 bytes (1.6217880249023438 Mb)
Ytrain_c           ndarray     495998: 495998 elems, type `int64`, 3967984 bytes (3.7841644287109375 Mb)
auc                function    <function roc_auc_score at 0x7f2f7d8e1ae8>
bst                Booster     <xgboost.core.Booster object at 0x7f2f7c887c18>
n_dim              int         260
n_train            int         708569
np                 module      <module 'numpy' from '/ho<...>kages/numpy/__init__.py'>
num_round          int         510
os                 module      <module 'os' from '/home/<...>da3/lib/python3.5/os.py'>
param              dict        n=10
pd                 module      <module 'pandas' from '/h<...>ages/pandas/__init__.py'>
plt                module      <module 'matplotlib.pyplo<...>es/matplotlib/pyplot.py'>
sns                module   

In [21]:
Xtest = pd.read_csv("./test/joint_accident_person_test_ext.csv", index_col=0)
ID= Xtest['ID'].astype(np.int64)
Xtest.drop('ID', axis= 1, inplace=True)
print("Load test samples")

Load test samples


In [22]:
uniqueID = np.unique(ID)

In [ ]:
uniqueID

In [23]:
dtest= xgb.DMatrix(data=Xtest.values, missing = NA)

In [24]:
print("Prediction")
preds = bst.predict(dtest)

Prediction


In [25]:
dtest.save_binary('./models/binary/dtest.buffer')


In [27]:
print("submit")
predict_df = pd.DataFrame(data={'ID': ID.values, 'prob': preds})
predict_df

submit


,ID,prob
0,0,0.171901
1,1,0.254055
2,1,0.593484
3,1,0.312187
4,2,0.191295
5,2,0.126673
6,2,0.141163
7,3,0.028505
8,3,0.024859
9,4,0.023092


In [28]:
predict_df.groupby('ID').mean()

,prob
ID,
0,0.171901
1,0.386575
2,0.153044
3,0.026682
4,0.023092
5,0.635964
6,0.085232
7,0.060965
8,0.286865


In [30]:
grouped_predict = predict_df.groupby('ID')
prediction = grouped_predict.mean()

In [32]:
prediction.index

Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,
                 8,      9,
            ...
            121055, 121056, 121057, 121058, 121059, 121060, 121061, 121062,
            121063, 121064],
           dtype='int64', name='ID', length=121065)

In [34]:
submit = pd.DataFrame(data={'ID': prediction.index, 'DRUNK_DR': prediction})
submit.to_csv('fars_submit.csv', index = False)